# Day 16

You can't read your train ticket, but you know what the 20 field represent, and you know what values are valid for the fields.

### Part 1

Determine the sum of all values on all scanned tickets that don't match any field (class).

In [1]:
f = open('day16.txt')
info = f.read().splitlines()
f.close()

In [2]:
info[:30]

['departure location: 32-209 or 234-963',
 'departure station: 47-64 or 83-967',
 'departure platform: 37-609 or 628-970',
 'departure track: 29-546 or 567-971',
 'departure date: 50-795 or 816-960',
 'departure time: 49-736 or 750-962',
 'arrival location: 48-399 or 420-967',
 'arrival station: 49-353 or 360-967',
 'arrival platform: 37-275 or 298-969',
 'arrival track: 40-119 or 127-954',
 'class: 35-750 or 760-968',
 'duration: 43-162 or 186-963',
 'price: 30-889 or 914-949',
 'route: 39-266 or 274-950',
 'row: 45-366 or 389-954',
 'seat: 42-765 or 772-955',
 'train: 30-494 or 518-957',
 'type: 48-822 or 835-973',
 'wagon: 32-330 or 342-951',
 'zone: 36-455 or 462-973',
 '',
 'your ticket:',
 '109,137,131,157,191,103,127,53,107,151,61,59,139,83,101,149,89,193,113,97',
 '',
 'nearby tickets:',
 '141,889,637,661,351,108,544,424,330,479,662,448,152,584,408,736,714,299,61,864',
 '692,855,843,361,21,265,678,716,347,531,56,792,492,656,727,848,149,596,887,862',
 '819,139,202,774,606,242,64

#### Parse puzzle input into three parts

In [3]:
# get class intervals
i = 0
classes = []
while info[i] != '':
    line = info[i]
    int1, int2 = line[line.index(':')+1:].split(' or ')
    classes.append(([int(x) for x in int1.split('-')], [int(x) for x in int2.split('-')]))
    i += 1
i += 2

# get my ticket information
myticket = [int(x) for x in info[i].split(',')]
print(myticket)
i += 3

# get scanned ticket information
tickets = []
while i < len(info):
    tickets.append([int(x) for x in info[i].split(',')])
    i += 1

[109, 137, 131, 157, 191, 103, 127, 53, 107, 151, 61, 59, 139, 83, 101, 149, 89, 193, 113, 97]


In [4]:
classes[:5]

[([32, 209], [234, 963]),
 ([47, 64], [83, 967]),
 ([37, 609], [628, 970]),
 ([29, 546], [567, 971]),
 ([50, 795], [816, 960])]

In [5]:
# flatten the classes for this part, ticket vals need to match any class
allclasses = [classes[j][i] for j in range(len(classes)) for i in range(2)]
allclasses[:5]

[[32, 209], [234, 963], [47, 64], [83, 967], [37, 609]]

In [6]:
# at first I was stupid and looking to count how many tickets contained an invalid value
invcount = 0
for t in tickets:
    invalid = False
    i = 0
    while i < len(t):
        if any([c[0] > t[i] or c[1] < t[i] for c in allclasses]):
            invalid = True
            break
        else:
            i += 1
    if invalid:
        invcount += 1
invcount

236

In [7]:
# actual Part 1 solution
err = 0
for t in tickets:
    for val in t:
        if any([c[0] <= val and c[1] >= val for c in allclasses]):
            continue
        else:
            err += val
err

19060

### Part 2

Now, remove all the invalid tickets, and use the valid tickets to determine which values correspond to which fields. All the tickets have the same order, including mine, so for example, perhaps the 3rd value on every ticket indicates the price.

Determine in which order the values on each ticket correspond to the available fields.

Then, find the product of the 6 "departure" values on your ticket (first 6).

In [8]:
# remove invalid
bad = []
for t in tickets:
    for val in t:
        if any([c[0] <= val and c[1] >= val for c in allclasses]):
            continue
        else:
            bad.append(t)
goodtix = [t for t in tickets if t not in bad]

In [9]:
avail = classes[:]   # copy classes to avail so we can modify it
order = {}
unclaimed = list(range(len(goodtix[0])))   # ticket indices we haven't matched yet

while unclaimed:
    for i in unclaimed:
        vals = [t[i] for t in goodtix]   # get i_th val from every ticket
        goodavailind = []
        for aind, a in enumerate(avail):
            if all([(a[0][0] <= v <= a[0][1]) or (a[1][0] <= v <= a[1][1]) for v in vals]):
                goodavailind.append(aind)
        if len(goodavailind) == 1:   # only want to claim a match if its the only possible match
            aind = goodavailind[0]
            order[aind] = i   # put match in final dict
            avail[aind] = [[-1,-1],[-1,-1]]   # prevent this class from being matched again
            unclaimed.remove(i)   # prevent the i_th val on each tix from being checked again
            break
                

In [10]:
order
# index of fields as given: index on each ticket the fields info can be found

{14: 7,
 8: 10,
 10: 15,
 17: 6,
 19: 12,
 7: 1,
 1: 11,
 2: 17,
 0: 5,
 4: 14,
 5: 13,
 3: 19,
 13: 18,
 16: 16,
 15: 3,
 6: 0,
 12: 9,
 18: 2,
 9: 4,
 11: 8}

In [11]:
ans = 1
for i in range(6):
    ans *= myticket[order[i]]
ans

953713095011